In [2]:
# Load the video data 
import numpy as np
import skvideo.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
from matplotlib.widgets import Slider, Button
from tqdm import tqdm
from queue import Queue
import scipy.optimize as opt
import pickle
import pandas as pd
import xarray as xr
from timeit import default_timer as timer
%matplotlib inline

In [3]:
filename = "sample.mp4" 
frames_to_load = 7273 # this is necessary because it seems to crash at the end of the video 

moviedata = skvideo.io.vread(filename, num_frames=frames_to_load)

crop = 50
test_frames = np.random.permutation(frames_to_load)[:100]

raw = moviedata[test_frames, crop:-crop, crop:-crop, 0]

[T, X, Y] = raw.shape
x = np.arange(-X/2, X/2)
y = np.arange(-Y/2, Y/2)
#print(np.size(x),np.size(y))
x2d, y2d = np.meshgrid(y, x)
radius = np.sqrt(x2d**2 + y2d**2)
print(T, X, Y)

100 284 284


In [4]:
def check_adjacent(i, j, data, status, q):
    # process adjacent pixels and add them to the queue if need be 
    if not status[i, j]:
        if data[i, j]:
            q.put([i, j])
        else: 
            status[i, j] = True

def find_connected_components(data):
    [X, Y] = data.shape
    curr = 0
    labels = np.zeros_like(data, dtype=int) - 1
    status = np.zeros_like(data, dtype=bool)
    q1 = Queue()
    q2 = Queue()
    for i in range(X):
        for j in range(Y):

            # skip this pixel if we already got it
            if status[i, j]:
                continue

            # if this pixel is false, flag it as processed and continue
            if not data[i, j]:
                status[i, j] = True
                continue
        
            # check adjacent pixels and add them to the queue if needed 
            check_adjacent(i, (j+1)%Y, data, status, q1)
            check_adjacent(i, (j-1)%Y, data, status, q1)
            check_adjacent((i+1)%X, j, data, status, q1)
            check_adjacent((i-1)%X, j, data, status, q1)
            labels[i, j] = curr
            status[i, j] = True
            # process the queue
            while not q1.empty():
                [a, b] = q1.get()
                check_adjacent(a, (b+1)%Y, data, status, q2)
                check_adjacent(a, (b-1)%Y, data, status, q2)
                check_adjacent((a+1)%X, b, data, status, q2)
                check_adjacent((a-1)%X, b, data, status, q2)
                labels[a, b] = curr
                status[a, b] = True
                while not q2.empty():
                    q1.put(q2.get())
            curr += 1

    return labels, curr-1



# let's do a steamlined version of this pipeline, to try to design maximally efficient filters

def std_n_particles(x):
    start = timer()
    filter_scale, threshfrac = x

    real_space_filter = np.exp(-2 * (radius / filter_scale)**2) * (1 - 2 * (radius / filter_scale)**4)

    fft_data = np.fft.fft2(raw, axes=[1, 2])
    fourier_filter = np.fft.fft2(np.fft.fftshift(real_space_filter))
    filtered = np.real(np.fft.ifft2(fft_data*fourier_filter))

    threshold = np.amax(filtered) * threshfrac

    thresh = np.copy(filtered)
    thresh[thresh < threshold] = 0
    thresh[thresh > 0] = 1
    binary = np.asarray(thresh, dtype=bool)

    particle = np.zeros_like(raw, dtype=int)
    n_particles = np.zeros(T, dtype=int)

    for t in tqdm(range(T)):
        l, n = find_connected_components(binary[t])
        particle[t] = l
        n_particles[t] = n

    vals, counts = np.unique(n_particles, return_counts=True) 
    end = timer()
    return vals, counts, end - start


In [39]:
scalearr = np.arange(2, 5, 0.25)
threshfracs = np.arange(0.2, 0.6, 0.025)
S = len(scalearr)
F = len(threshfracs)
res = []
for i, s in enumerate(scalearr):
    curr = []
    for j, f in enumerate(threshfracs): 
        print(f"s = {s}, f = {f}")
        curr.append(std_n_particles((s, f)))
    res.append(curr)
    

s = 2.0, f = 0.2


100%|██████████| 100/100 [00:38<00:00,  2.60it/s]


s = 2.0, f = 0.225


100%|██████████| 100/100 [00:17<00:00,  5.68it/s]


s = 2.0, f = 0.25


...

100%|██████████| 100/100 [00:01<00:00, 56.85it/s]


s = 4.75, f = 0.5499999999999999


100%|██████████| 100/100 [00:01<00:00, 59.51it/s]


s = 4.75, f = 0.575


100%|██████████| 100/100 [00:01<00:00, 62.17it/s]


In [42]:
# optimizationresults = {"scalearr": scalearr, "threshfracs": threshfracs, "res": res}
# pickle.dump(optimizationresults, open("optimizationresults.p", "wb"))

In [3]:

optimizationresults = pickle.load(open("optimizationresults.p", "rb"))
res = optimizationresults["res"]
scalearr = np.arange(2, 5, 0.25)
threshfracs = np.arange(0.2, 0.6, 0.025)
S = len(scalearr)
F = len(threshfracs)

In [16]:
avgparticles = np.zeros((S, F))
stdparticles = np.zeros((S, F))
times = np.zeros((S, F))
for i, s in enumerate(scalearr):
    for j, f in enumerate(threshfracs):
        avgparticles[i, j] = np.sum(res[i][j][0] * res[i][j][1])/np.sum(res[i][j][1])
        stdparticles[i, j] = np.sqrt(np.sum(
            res[i][j][1] * (res[i][j][0]-avgparticles[i, j])**2)/np.sum(res[i][j][1]))
        times[i, j] = res[i][j][2]
        
avgparticles = np.transpose(avgparticles)
stdparticles = np.transpose(stdparticles)

xbest = 3.125 + np.array([0, 0.25, 0.25, 0, 0])
ybest = 0.2875 + np.array([0, 0, 0.025, 0.025, 0])

plt.figure(figsize=(4,8))
plt.title("Particle detection accuracy and efficiency vs. filter scale and threshold fraction")
plt.subplot(2, 1, 1)
# bounds = np.array([0, 191, 192, 193, 194, 195, 196, 197, 198, 199, 300])
# plt.pcolormesh(threshfracs, scalearr, avgparticles, norm=colors.BoundaryNorm(boundaries=bounds, ncolors=256))
plt.pcolormesh(scalearr, threshfracs, avgparticles, vmin=190, vmax=200)
plt.plot(xbest, ybest, color="red")
plt.text(3.5, 0.3, "Optimum", color="red", verticalalignment="center")
plt.xlabel("filter scale (pix)")
plt.ylabel("threshold fraction")
# plt.colorbar(label="mean # particles (very nonlinear colorbar!)")
plt.colorbar(label="mean # particles")
plt.subplot(2, 1, 2)
plt.pcolormesh(scalearr, threshfracs, stdparticles, norm=colors.LogNorm())
plt.plot(xbest, ybest, color="red")
plt.text(3.5, 0.3, "Optimum", color="red", verticalalignment="center")
plt.xlabel("filter scale [pix]")
plt.ylabel("threshold fraction")
plt.colorbar(label="std # particles")
plt.savefig("optimizationresults.pdf")
plt.show()

In [54]:
# what is the actual value of threshold for the max threshfrac and filterscale here? 


filter_scale, threshfrac = (3.25, 0.3)

real_space_filter = np.exp(-2 * (radius / filter_scale)**2) * (1 - 2 * (radius / filter_scale)**4)

fft_data = np.fft.fft2(raw, axes=[1, 2])
fourier_filter = np.fft.fft2(np.fft.fftshift(real_space_filter))
filtered = np.real(np.fft.ifft2(fft_data*fourier_filter))

threshold = np.amax(filtered) * threshfrac
print(threshold)

240.6991028224026


In [ ]:
# so for our data run, let's us filter_scale 3.25 and threshold 240!